In [102]:
from datetime import datetime
from datetime import date
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import math
import pandas as pd

In [103]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

Get the list of spaces. We don't know how many spaces we have, so we have to cycle and only stop when the list is not filled with 1000 rows

In [89]:
spaces=[]
spaces = snapshot.Query.spaces(
  orderBy='created',
  orderDirection='asc',
  first=1000,
)
spaces_list = sg.query_df([
    spaces.id
    ]
  )


In [90]:
spaces2 = []
n=0
t=2
while n<1:
  spaces2 = snapshot.Query.spaces(
    orderBy='created',
    orderDirection='asc',
    first=1000,
    skip = 1000*t
  )
  spaces_list2 = sg.query_df([
    spaces2.id
    ]
  )
  spaces_list = pd.concat([spaces_list,spaces_list2])
  t=t+1
  l=len(spaces_list2)
  print(t,l)
  if len(spaces_list2)<1000: n=1

olympus_coord=spaces_list.index[spaces_list['spaces_id'] == 'olympusdao.eth'].tolist()


3 1000
4 1000
5 1000
6 1000
7 1000
8 1000
9 1000
10 17


In [91]:
numspaces = len(spaces_list)
numspaces

8017

We get the list of elections  for each snapshot. Note the choice of vote only carries the ID not the question

In [92]:
space_name = spaces_list.iat[1,0]
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='asc',
  first=1000,
  where=[
    snapshot.Proposal.space == space_name,
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

proposals_snapshots = sg.query_df([
    proposals.title,
    proposals.id,
    proposals.body,
    proposals.scores,
    proposals.scores_total
])

proposals_snapshots.insert(0,'dao',space_name)

proposals_snapshots

,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,fabien.eth,Test,QmbGQCSWXqJwUUksSJ5UbYgjnrpn25aJNyuLFUA48wYg8Q,**Bold test** Lost of text\n\n# Proposal: Test...,0.000000,0.000000
1,fabien.eth,Test,QmbGQCSWXqJwUUksSJ5UbYgjnrpn25aJNyuLFUA48wYg8Q,**Bold test** Lost of text\n\n# Proposal: Test...,0.000000,0.000000
2,fabien.eth,Test proposal,QmPDvn4oamr1ZxdTXH3pCKn4jpxTJUv3yMVrCBxfgDmyEA,This is a test,0.000000,12.503688
3,fabien.eth,Test proposal,QmPDvn4oamr1ZxdTXH3pCKn4jpxTJUv3yMVrCBxfgDmyEA,This is a test,12.503688,12.503688
4,fabien.eth,Test proposal,QmWqUBX1jgWzs8A8hMAV2bSk8jm2HQ3wdq4FWgt4QR5U15,This is a test,18.000000,29.000000
...,...,...,...,...,...,...
870,fabien.eth,Test webhook,0xa66466451daeca21e2d50188d3b72ae6cd36626ca44f...,,52.784172,55.784172
871,fabien.eth,Test webhook,0xa66466451daeca21e2d50188d3b72ae6cd36626ca44f...,,1.000000,55.784172
872,fabien.eth,Test for EIP-1271 on Optimism,0xe0fd8430e73de3b4790648b6edb9bba0b3255c3fe7fa...,,0.000000,4445.481307
873,fabien.eth,Test for EIP-1271 on Optimism,0xe0fd8430e73de3b4790648b6edb9bba0b3255c3fe7fa...,,2018.027507,4445.481307


In [93]:
space_name = spaces_list.iat[2,0]

proposals2 = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=1000,
  where=[
    snapshot.Proposal.space == space_name,
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

proposals_snapshots2 = sg.query_df([
    proposals2.title,
    proposals2.id,
    proposals2.body,
    proposals2.scores,
    proposals2.scores_total
])

proposals_snapshots2.insert(0,'dao',space_name)

proposals_snapshots2

,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,ichi.eth,Volatility Increase in Vault IRR Deposits,0xb6acfcda2b1c19a231f7df2c6cd7bcd5654a8be989e2...,Summary:\nPropose an increase in vault rewards...,43073.307504,43073.307504
1,ichi.eth,Volatility Increase in Vault IRR Deposits,0xb6acfcda2b1c19a231f7df2c6cd7bcd5654a8be989e2...,Summary:\nPropose an increase in vault rewards...,0.000000,43073.307504
2,ichi.eth,ICHI Proposal for Bancor V3,0xefd26a12611d809411277a23ab59d41c9b56703a2faa...,"Summary:\n-Provide 50,000 BNT worth of Legacy ...",43073.307504,43073.307504
3,ichi.eth,ICHI Proposal for Bancor V3,0xefd26a12611d809411277a23ab59d41c9b56703a2faa...,"Summary:\n-Provide 50,000 BNT worth of Legacy ...",0.000000,43073.307504
4,ichi.eth,ICHI Vaults Proposal,0x9e724ac779d020df40016be159a4a8062965a84fee63...,Summary:\nProposal allocating $ICHI to the fol...,94158.348758,94210.981940
...,...,...,...,...,...,...
105,ichi.eth,Should we launch a community website on ichi.org?,Qmbj2pu23vbNPJGSMt1bgJjK4YGxcDMjeqggyLNHgDbG5V,Proposal from samwisegardener in the ichi disc...,0.000000,109450.440901
106,ichi.eth,test,QmPmUra8QTkymdJNKT2QjYUtJtXpVY6wE4QT66genw9DQZ,test,9039.392908,9039.392908
107,ichi.eth,test,QmPmUra8QTkymdJNKT2QjYUtJtXpVY6wE4QT66genw9DQZ,test,0.000000,9039.392908
108,ichi.eth,Test?,QmRRJ8uwthPoTCmGxk2TANdhDnppECvKBcaz55FywySRP4,Test - yes or no,0.000000,0.000000


In [94]:
proposals3 = []
t=int(input('set starting point for DAOs'))
limit=int(input('set limit for DAOs to pull (-1 for all records)'))+2
if limit==-1:
  limit = numspaces

while t<=limit: ##numspaces when out of testing
  space_name = spaces_list.iat[t,0]
  proposals3 = snapshot.Query.proposals(
    orderBy='created',
    orderDirection='desc',
    first=1000,
    skip = 1000*t,
    where=[
      snapshot.Proposal.space == space_name,
      snapshot.Proposal.state == 'closed'
      ]
  )
  proposals_snapshots3 = sg.query_df([
    proposals3.title,
    proposals3.id,
    proposals3.body,
    proposals3.scores,
    proposals3.scores_total
])
  proposals_snapshots3.insert(0,'dao',space_name)
  proposals_snapshots = pd.concat([proposals_snapshots,proposals_snapshots2,proposals_snapshots3])
  t=t+1
  l=len(proposals_snapshots3)
  if t%10 == 0: print (t,space_name,l)
  if t>1150: print (t,space_name,l)
  if len(proposals_snapshots3)>0:
    print(t,space_name,l)

proposals_snapshots

,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,fabien.eth,Test,QmbGQCSWXqJwUUksSJ5UbYgjnrpn25aJNyuLFUA48wYg8Q,**Bold test** Lost of text\n\n# Proposal: Test...,0.000000,0.000000
1,fabien.eth,Test,QmbGQCSWXqJwUUksSJ5UbYgjnrpn25aJNyuLFUA48wYg8Q,**Bold test** Lost of text\n\n# Proposal: Test...,0.000000,0.000000
2,fabien.eth,Test proposal,QmPDvn4oamr1ZxdTXH3pCKn4jpxTJUv3yMVrCBxfgDmyEA,This is a test,0.000000,12.503688
3,fabien.eth,Test proposal,QmPDvn4oamr1ZxdTXH3pCKn4jpxTJUv3yMVrCBxfgDmyEA,This is a test,12.503688,12.503688
4,fabien.eth,Test proposal,QmWqUBX1jgWzs8A8hMAV2bSk8jm2HQ3wdq4FWgt4QR5U15,This is a test,18.000000,29.000000
...,...,...,...,...,...,...
105,ichi.eth,Should we launch a community website on ichi.org?,Qmbj2pu23vbNPJGSMt1bgJjK4YGxcDMjeqggyLNHgDbG5V,Proposal from samwisegardener in the ichi disc...,0.000000,109450.440901
106,ichi.eth,test,QmPmUra8QTkymdJNKT2QjYUtJtXpVY6wE4QT66genw9DQZ,test,9039.392908,9039.392908
107,ichi.eth,test,QmPmUra8QTkymdJNKT2QjYUtJtXpVY6wE4QT66genw9DQZ,test,0.000000,9039.392908
108,ichi.eth,Test?,QmRRJ8uwthPoTCmGxk2TANdhDnppECvKBcaz55FywySRP4,Test - yes or no,0.000000,0.000000


In [95]:
total_props = int(len(proposals_snapshots))
chunks = math.ceil(total_props/500)
print(total_props,'elections. Pulling in', chunks,'sets ')

1205 elections. Pulling in 3 sets 


We are going to use this to timestamp the filenames

In [96]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
  )


In [97]:
##spit out the file!
file = input('Selet a folder') ##enter your file path here - the file is in the repo "summary_stats.csv".
path =file+'/votes_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(proposals_snapshots))+'_snapshots.csv'
proposals_snapshots.to_csv(path, index = False)
print('file saved: '+ path)

file saved: C:\Users\ianfe\Dropbox\Documents\Olympus\Governance/votes_table_Jul-13-2022_1205_snapshots.csv


In [98]:
proposal_id = proposals_snapshots.iloc[1,2]
voting_snapshots_list = sg.query_df([
  vote_tracker.id,
  vote_tracker.voter,
  vote_tracker.created,
  vote_tracker.choice,
  vote_tracker.vp
])
print(proposal_id)
voting_snapshots_list

QmbGQCSWXqJwUUksSJ5UbYgjnrpn25aJNyuLFUA48wYg8Q


,votes_id,votes_voter,votes_created,votes_choice,votes_vp
0,0x1df60184b9ff3f810b2444a39682e06609509946aaf4...,0xcdcFd445acAf0C3494Bb84A551fdAC3F1d85B1A4,1638197107,2,83629.098701
1,0xc612cb181227a6b77a9bc000e1c531b1f7c30b17b634...,0xd6D5affd829AaB689b74Ff016287dE16c45f433F,1638197056,2,45326.801030
2,0xeaa20af9590a34c258d7f7024eb6931c0f703d7bcca1...,0x8BF0D893Af34Bbc41cE80Fe2f86b8c5298e6ED0f,1638196896,2,23678.757635
3,0x0641ab489ec89ff3cf1de80daab90d740aba51acfc98...,0x47Ea9C5E604Fb483412B0de8Ff5AE4c82B02D065,1638196760,2,19737.545667
4,0x2f546d4a1e4d6bc586b860e7d2b6bddedbffe144158c...,0xD5b0A2a96907344A537Eb809A0B75b895492E2f8,1638196708,2,29487.693485
5,0x7587b79dd4c8e3f488e24923a0ec6afc055931dfad70...,0x1DAb06752D40796847f54dA480f4F9f38052B108,1638194757,3,5816.058240
6,0x7856b90dd21c12de5cd6b41a247ba5cacba484020b44...,0x210c09B2390ee03Ee86dD8bb736BDA2F31AaB57F,1638191883,3,54774.488778
7,0x43274540543d9d9678323799ccb7dbabfb904eecd687...,0xCeFC6e5826dFA7a81F0AbaB83C46E7173Ea014C3,1638187373,1,49735.262654
8,0xf87870513af9a54336635826d81953e5f532088d7053...,0x8f02319F577e17A71864B78dfbD50C55ACBeDeDE,1638182926,4,33.095298
9,QmSca8fuEBwG7Moc6thJAcoP3CXUrQBhheFPQuXs5gd9mW,0xF61D614B5fEF9d74a75e765dE8c32f140Ab7D177,1638181182,3,14793.543732


In [99]:
voting_snapshots_list['Proposal'] = proposal_id
print(voting_snapshots_list)

                                             votes_id  \
0   0x1df60184b9ff3f810b2444a39682e06609509946aaf4...   
1   0xc612cb181227a6b77a9bc000e1c531b1f7c30b17b634...   
2   0xeaa20af9590a34c258d7f7024eb6931c0f703d7bcca1...   
3   0x0641ab489ec89ff3cf1de80daab90d740aba51acfc98...   
4   0x2f546d4a1e4d6bc586b860e7d2b6bddedbffe144158c...   
5   0x7587b79dd4c8e3f488e24923a0ec6afc055931dfad70...   
6   0x7856b90dd21c12de5cd6b41a247ba5cacba484020b44...   
7   0x43274540543d9d9678323799ccb7dbabfb904eecd687...   
8   0xf87870513af9a54336635826d81953e5f532088d7053...   
9      QmSca8fuEBwG7Moc6thJAcoP3CXUrQBhheFPQuXs5gd9mW   
10  0x076aa10733b044a19c3528181ad5fe386fc57533d4a7...   
11  0x6e32e151b4ec5d608fac364dce3d9fffa84973ad9fb3...   
12  0x643f10b75cbfa6ba05981886c4aabdd1f9fc570b6e5f...   
13  0x243e3e49855271cef6d240f45b1bc4fa08b43f2f3cd4...   
14     QmYp9hvBStGn5YHnrptaxRqhJgbYKHF4kP4JPuT69KaYt6   
15  0x0224e6982f183898bb22490a18b5c744715f2eae1245...   
16  0xc71915aa696acbce3364ee5a9

In [101]:
n=2
iteration =1
print('pull started')
endpoint = min(5000,total_props)
while iteration <= chunks:
  while n<=endpoint: ##total_props:
    proposal_id = proposals_snapshots.iloc[n,2]
    ##if n%10 == 0: print (n,proposal_id)
    n=n+1
    vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=10000,
    where=[
      snapshot.Vote.proposal == proposal_id
    ])
    voting_snapshots_list2 = sg.query_df([
      vote_tracker.id,
      vote_tracker.voter,
      vote_tracker.created,
      vote_tracker.choice,
      vote_tracker.vp
    ])
    voting_snapshots_list2['Proposal'] = proposal_id
    voting_snapshots_list = pd.concat([voting_snapshots_list,voting_snapshots_list2])
    iteration = iteration +1
    iter_progress = 100*round(n/5000,4)
    remaining = (total_props-n)
    if n%500==0:
      print(n, 'proposals pulled.',remaining, 'remain in this chunk','\nProgress in this chunk:', iter_progress,'%')

      ##now we save that part in its file
  path =file+'/votes_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(voting_snapshots_list))+'_proposals_part_'+iteration+'.csv'
  voting_snapshots_list.to_csv(path, index = False)
  chunk_progress = 100*round(remaining/total_props,2)
  print('file saved:',path,
        'progress in whole download', chunk_progress,'%')




pull started


KeyboardInterrupt: 

And finally, the body of each election's choices. Not critical but useful to have.

Still needs work! Having a tough time automating pulling all those choices into a single table.

In [117]:
proposals_choices = sg.query(proposals.choices)
proposals_choices

TypeError: 'FieldPath' object is not iterable

In [118]:
proposals_choices = sg.query(proposals.choices)

TypeError: 'FieldPath' object is not iterable

In [119]:
##placing 25 options just in case. Could be more, but I hope not
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_1', 'option_2', 'option_3', 'option_4', 'option_5','option_6','option_7','option_8','option_9','option_10','option_11','option_12','option_13','option_14','option_15','option_16','option_17','option_18','option_19','option_20','option_21','option_22','option_24','option_24','option_25]']

SyntaxError: '(' was never closed (1609945154.py, line 2)

In [ ]:
dao_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [ ]:
dao_governance_view

In [ ]:
##spit out the file!
path =file+'/votes_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(voting_snapshots_list))+'_proposals.csv'
voting_snapshots_list.to_csv(path, index = False)
print('file saved')